## Name: Vaibhav Bichave

## Object detection using Transfer Learning of CNN architectures for the given (image dataset) using the below steps:
    a.Load in a pre-trained CNN model trained on a large dataset
    b.Freeze parameters (weights) in model's lower convolutional layers
    c.Add custom classifier with several layers of trainable parameters to model
    d.Train classifier layers on training data available for task
    e.Fine-tune hyper parameters and unfreeze more layers as needed 

In [1]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import UpSampling2D,Dense,Input,Flatten
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [2]:
(X_train,y_train),(X_test,y_test) = cifar10.load_data()

In [3]:
X_train = X_train/255.0
X_test = X_test/255.0

In [4]:
y_train = y_train.flatten()
y_test = y_test.flatten()

In [5]:
base_model = InceptionV3(include_top=False,weights='imagenet', 
                         input_shape=(128, 128, 3),pooling='avg')

for layer in base_model.layers: 
   layer.trainable = False

In [6]:
inputlayer = Input(shape=(32, 32, 3))
model = Sequential([
    inputlayer,
    UpSampling2D((2,2)),
    UpSampling2D((2,2)),
    base_model,
    Dense(128, activation='relu'),
    Dense(32, activation='relu'),
    Flatten(),
    Dense(10, activation='softmax')   
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 up_sampling2d (UpSampling2D  (None, 64, 64, 3)        0         
 )                                                               
                                                                 
 up_sampling2d_1 (UpSampling  (None, 128, 128, 3)      0         
 2D)                                                             
                                                                 
 inception_v3 (Functional)   (None, 2048)              21802784  
                                                                 
 dense (Dense)               (None, 128)               262272    
                                                                 
 dense_1 (Dense)             (None, 32)                4128      
                                                                 
 flatten (Flatten)           (None, 32)                0

In [7]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(optimizer='adam',loss=SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [8]:
H = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=1)

1563/1563 [==============================] - 1302s 829ms/step - loss: 0.8759 - accuracy: 0.7081 - val_loss: 0.7293 - val_accuracy: 0.7526


In [10]:
prediction = model.predict(X_test)

In [11]:
from sklearn.metrics import classification_report
print(classification_report(y_test,prediction.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.76      0.73      0.75      1000
           1       0.83      0.82      0.83      1000
           2       0.77      0.71      0.74      1000
           3       0.69      0.52      0.59      1000
           4       0.75      0.67      0.71      1000
           5       0.66      0.79      0.72      1000
           6       0.72      0.83      0.77      1000
           7       0.77      0.82      0.79      1000
           8       0.81      0.80      0.80      1000
           9       0.78      0.85      0.81      1000

    accuracy                           0.75     10000
   macro avg       0.75      0.75      0.75     10000
weighted avg       0.75      0.75      0.75     10000

